In [93]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import spacy
import nltk

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import CountVectorizer

In [94]:
from sklearn.model_selection import train_test_split
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/cathal/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../raw_data/concat.csv')

df.info()

/tmp/ipykernel_3196/4288383160.py:6: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/concat.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575716 entries, 0 to 575715
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      575716 non-null  int64  
 1   name            575716 non-null  object 
 2   date            435396 non-null  object 
 3   recommendation  434891 non-null  object 
 4   review          574197 non-null  object 
 5   playing_hours   434891 non-null  float64
 6   source          575716 non-null  object 
 7   stars           140825 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 35.1+ MB


In [63]:
df = df[df['review'].notnull()]

df.head()

,Unnamed: 0,name,date,recommendation,review,playing_hours,source,stars
0,0,Expansion - Hearts of Iron IV: Man the Guns,2019-02-10,Recommended,&gt Played as German Reich&gt Declare war on B...,578.0,steam,NaN
1,1,Expansion - Hearts of Iron IV: Man the Guns,2019-02-10,Recommended,yes.,184.0,steam,NaN
2,2,Expansion - Hearts of Iron IV: Man the Guns,2019-02-07,Recommended,Very good game although a bit overpriced in my...,892.0,steam,NaN
3,3,Dead by Daylight,2018-06-14,Recommended,Out of all the reviews I wrote This one is pro...,676.0,steam,NaN
4,4,Dead by Daylight,2017-06-20,Recommended,Disclaimer I survivor main. I play games for f...,612.0,steam,NaN


In [64]:
dfstars = df[df['source'] != 'steam'] 
dfstars['stars'].value_counts()

5.0    106924
4.0     22460
3.0      5923
1.0      2961
2.0      2554
Name: stars, dtype: int64

In [65]:
df3 = dfstars[dfstars['stars']==3]
df3.head(20)

,Unnamed: 0,name,date,recommendation,review,playing_hours,source,stars
435401,5,2-speed-it,NaN,NaN,I was disappointed because the name is mislead...,NaN,coursera,3.0
435408,12,2-speed-it,NaN,NaN,"Good content, but the course setting does (at ...",NaN,coursera,3.0
435416,20,2-speed-it,NaN,NaN,"The course content is quite good, though it co...",NaN,coursera,3.0
435489,93,3d-printing-applications,NaN,NaN,I'll start by saying that this course gives a ...,NaN,coursera,3.0
435493,97,3d-printing-applications,NaN,NaN,It should be merged with course 1,NaN,coursera,3.0
435502,106,3d-printing-applications,NaN,NaN,Full Individuals speaking,NaN,coursera,3.0
435503,107,3d-printing-applications,NaN,NaN,It should be less repetitive and full of usefu...,NaN,coursera,3.0
435506,110,3d-printing-applications,NaN,NaN,it could be easily compressed in a two-weeks c...,NaN,coursera,3.0
435507,111,3d-printing-applications,NaN,NaN,good introduction into the applications of 3D ...,NaN,coursera,3.0
435509,113,3d-printing-applications,NaN,NaN,This session was too long and a little boring....,NaN,coursera,3.0


In [66]:
df_dumb = df[df['source'] != 'steam'] 
             
# df_dumb = df_dumb[df_dumb['stars'] != 3]

#here we are filtering the data base to only have sources whihc are not steam until stream has non english removed
#& we are removing 3 stars as they are hard tp convent to binary CANCEL

In [67]:
df_dumb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140822 entries, 434891 to 575715
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      140822 non-null  int64  
 1   name            140822 non-null  object 
 2   date            505 non-null     object 
 3   recommendation  0 non-null       object 
 4   review          140822 non-null  object 
 5   playing_hours   0 non-null       float64
 6   source          140822 non-null  object 
 7   stars           140822 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 9.7+ MB


In [68]:
df_dumb['recommendation'] = df_dumb['stars'].apply(lambda row:1 if row > 3 else 0)

#1 is recommended
#0 is not recommended
#concatenate with steam / as steam cannot go through this filter

/tmp/ipykernel_3196/302876873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dumb['recommendation'] = df_dumb['stars'].apply(lambda row:1 if row > 3 else 0)


In [69]:
df_dumb.head()

,Unnamed: 0,name,date,recommendation,review,playing_hours,source,stars
434891,0,Wyndham Capital Mortgage,10-04-2017,1,"Great job, Wyndham Capital! Each person was pr...",NaN,bank,5.0
434892,1,Wyndham Capital Mortgage,10-02-2017,1,Matthew Richardson is professional and helpful...,NaN,bank,5.0
434893,2,Wyndham Capital Mortgage,21-08-2017,1,We had a past experience with Wyndham Mortgage...,NaN,bank,5.0
434894,3,Wyndham Capital Mortgage,17-12-2017,1,We have been dealing with Brad Thomka from the...,NaN,bank,5.0
434895,4,Wyndham Capital Mortgage,27-05-2016,1,I can't express how grateful I am for the supp...,NaN,bank,5.0


In [70]:
df_dumb.shape

(140822, 8)

In [71]:
df_dumb['recommendation'].value_counts()

1    129384
0     11438
Name: recommendation, dtype: int64

In [72]:
df_model = df_dumb[['recommendation','review']]

In [73]:
X = df_model[['review']]
y = df_model['recommendation']
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.20, random_state=42)

In [74]:
df_model.dtypes

recommendation     int64
review            object
dtype: object

In [75]:
df_model.

SyntaxError: invalid syntax (11084182.py, line 1)

In [76]:


def remove_punctuations(text):
    punctuations = string.punctuation
    for punctuation in punctuations:
        text = text.replace(punctuation, '')
    return text

def lowercase(text):
    text = text.lower()
    return text

def remove_num(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text


# nlp = spacy.load("en_core_web_sm")
# def lemmatize(text):
#     text = ' '.join(token.lemma_ for token in nlp(text))
#     return text

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text) 
    lemmatized = [lemmatizer.lemmatize(word) for word in word_tokens]

    return ' '.join(lemmatized)

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    word_tokens = [word for word in word_tokens if not word in stop_words]
    return ' '.join(word_tokens)

In [79]:
df_model['review'] = df_model['review'].apply(remove_punctuations).apply(lowercase).apply(remove_num).apply(remove_stopwords)
# need to add lemmatizer not working

/tmp/ipykernel_3196/1222423747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['review'] = df_model['review'].apply(remove_punctuations).apply(lowercase).apply(remove_num).apply(remove_stopwords)


In [96]:
df_model['review'] = df_model['review'].apply(lemmatize)

/tmp/ipykernel_3196/2982901621.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['review'] = df_model['review'].apply(lemmatize)


In [97]:
for i in range(2):
    print (df_model['review'].iloc[i]) 

great job wyndham capital person professional helped u move refinance process smoothly thank
matthew richardson professional helpful helped u find correct product mortgage thank much excellent service matthew


In [98]:
df_model.head()

,recommendation,review
434891,1,great job wyndham capital person professional ...
434892,1,matthew richardson professional helpful helped...
434893,1,past experience wyndham mortgage would without...
434894,1,dealing brad thomka beginning started stressfu...
434895,1,cant express grateful support zach provided fa...


In [105]:
X = df_model['review']
y = df_model['recommendation']
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.20, random_state=42)

In [110]:
vectorizer = CountVectorizer()

X1_train = vectorizer.fit_transform(X_train)


y1 = y_train

nb_model = MultinomialNB()

cv_results = cross_validate(
    nb_model,
    X1_train,
    y1,
    scoring = ['f1','accuracy','recall', 'precision'],
    cv = 5)


from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(ngram_range = (2,2))

X2_train = tf_idf_vectorizer.fit_transform(X_train)

nb_model2 = MultinomialNB()

cv_results2 = cross_validate(
    nb_model2,
    X2_train,
    y1,
    scoring = ['f1','accuracy','recall', 'precision'],
    cv = 5)


# print(f" Model1: {cv_results['test_score'].mean()} Model2: {cv_results2['test_score'].mean()}")

In [111]:
cv_results

{'fit_time': array([0.03483939, 0.02989459, 0.04559517, 0.0380218 , 0.03375268]),
 'score_time': array([0.03998685, 0.02942443, 0.03272009, 0.03753424, 0.03686452]),
 'test_f1': array([0.95704052, 0.95682798, 0.95620752, 0.95709192, 0.95653423]),
 'test_accuracy': array([0.92042429, 0.92002485, 0.91877857, 0.92042075, 0.91939994]),
 'test_recall': array([0.96450476, 0.96440645, 0.9648894 , 0.96575872, 0.96503429]),
 'test_precision': array([0.94969092, 0.94936769, 0.94768049, 0.94857929, 0.94818259])}

In [112]:
cv_results2

{'fit_time': array([0.07916737, 0.06989098, 0.07408786, 0.08148932, 0.10857749]),
 'score_time': array([0.04780149, 0.0498476 , 0.04031682, 0.03746414, 0.07737947]),
 'test_f1': array([0.95803257, 0.9578755 , 0.95784944, 0.95787355, 0.95782924]),
 'test_accuracy': array([0.91949228, 0.91918161, 0.91913364, 0.91917802, 0.91908925]),
 'test_recall': array([0.99990341, 0.99990341, 0.99980682, 0.99985511, 0.99985511]),
 'test_precision': array([0.91952747, 0.91923811, 0.91927176, 0.91927534, 0.91919371])}

In [114]:
df_test = pd.DataFrame()
df_test['review'] = X_test
df_test['recommended']=y_test

In [115]:
df_test.head()

,review,recommended
500853,excellent course beginner must programming mat...,1
468986,awesome learn new thing,1
527721,great perspective multiple technique,1
551302,great teachergreat teaching methodologyinvalua...,1
529550,teacher explains well concept behind objective...,1


In [ ]:
# You're looking for stratification. Why?

# There's a parameter stratify in method train_test_split to which you can give the labels list e.g. :

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                     stratify=y, 
#                                                     test_size=0.2)

# Classification report might be what we want to understand how good/bad the model is at predicting the 
